## AgentCore Evaluations - evaluator 생성하기

이 튜토리얼에서는 AgentCore Evaluations의 내장 및 커스텀 메트릭에 대해 배웁니다.
각 유형을 언제 사용해야 하는지, 그리고 특정 요구사항에 맞는 커스텀 evaluator를 생성하는 방법을 배웁니다.

### 학습 내용
- 내장 evaluator와 사용 사례 이해하기
- 특수한 요구사항을 위한 커스텀 evaluator 생성하기
- Agent에 적합한 평가 방식 선택하기

### 튜토리얼 세부사항

| Information         | Details                                                                        |
|:--------------------|:-------------------------------------------------------------------------------|
| Tutorial type       | Create custom evaluatior metric                                                |
| LLM model           | Anthropic Claude Haiku 4.5                                                     |
| Tutorial components | Listing built-in evaluators, creating a custom evaluator metric                |
| Tutorial vertical   | Cross-vertical                                                                 |
| Example complexity  | Easy                                                                           |
| SDK used            | Amazon Bedrock AgentCore Starter toolkit                                       |

## Evaluator 유형

### 내장 evaluator

AgentCore는 Large Language Model(LLM)을 판단자로 사용하여 agent 성능을 평가하는 13개의 사전 구성된 evaluator를 제공합니다. 이러한 evaluator는 신중하게 제작된 프롬프트 템플릿과 사전 선택된 evaluator model을 함께 제공하여 다양한 사용 사례에서 평가 기준을 표준화합니다. 바로 사용할 수 있으며 시작하기 위해 추가 구성이 필요하지 않습니다.

이러한 evaluator는 4개의 그룹으로 나뉩니다:

- **Response quality**: 각 턴에서 agent가 예상대로 작동하는지 판단하는 데 도움이 되는 evaluator입니다. trace 기반으로 작동하며 각 사용자-agent 상호작용을 평가합니다.
- **Task completion**: 이 카테고리에는 세션 전체를 평가하는 하나의 evaluator(Goal success)가 있습니다. 멀티턴 대화에서 이 evaluator는 사용자 목표가 완료되었는지, 최종 결과가 달성되었는지 판단하는 데 도움이 됩니다. Agent가 후속 질문을 하는 상황에서 이 evaluator는 요청된 작업이 실제로 완료되었는지 이해하는 데 필수적입니다.
- **Tool level**: tool 호출이 얼마나 성공적인지 이해하는 데 도움이 되는 evaluator입니다. tool 레벨에서 agent의 tool 및 파라미터 선택의 정확성을 측정합니다. Agent가 단일 턴에서 두 개 이상의 다른 tool을 호출하는 경우, 각각은 trace에서 자체 메트릭을 갖습니다.
- **Safety**: agent가 유해하거나 개인 또는 그룹에 대해 고정관념적 일반화를 하고 있는지 감지하는 evaluator입니다.

내장 메트릭을 사용할 때는 프롬프트부터 model까지 모든 것이 자동으로 처리됩니다. 즉, 모든 사용자에게 일관되고 신뢰할 수 있는 평가를 유지하기 위해 evaluator를 수정할 수 없습니다. 그러나 내장 메트릭을 기반으로 자체 메트릭을 생성할 수 있습니다. 이를 위해 내장 evaluator의 **Prompt Templates**를 제공합니다.

### 커스텀 evaluator

커스텀 evaluator는 LLM을 기본 판단자로 활용하면서 평가 프로세스의 모든 측면을 정의할 수 있도록 하여 최대한의 유연성을 제공합니다. 커스텀 evaluator에서 다음을 사용자 정의할 수 있습니다:

- **Evaluator model**: 평가 요구사항에 가장 적합한 LLM 선택
- **Evaluation prompts**: 사용 사례에 특화된 평가 지침 작성
- **Scoring schema**: 조직의 메트릭에 맞는 점수 시스템 설계

### Agent에서 AgentCore Observability로 trace 생성하기

AgentCore Observability는 [OpenTelemetry (OTEL)](https://opentelemetry.io/) trace를 기반으로 상세한 실행 데이터를 캡처하고 구조화하여 호출 중 agent 동작에 대한 포괄적인 가시성을 제공합니다. AgentCore는 [AWS Distro for OpenTelemetry (ADOT)](https://aws-otel.github.io/)를 사용하여 다양한 agent 프레임워크에서 여러 유형의 OTEL trace를 계측합니다.

Agent가 AgentCore Runtime에서 호스팅되는 경우(이 튜토리얼의 agent처럼), AgentCore Observability 계측은 최소한의 구성으로 자동으로 이루어집니다. `requirements.txt`에 `aws-opentelemetry-distro`를 포함하기만 하면 AgentCore Runtime이 OTEL 구성을 자동으로 처리합니다. Agent가 AgentCore Runtime에서 실행되지 않는 경우, AgentCore Observability에서 사용할 수 있도록 ADOT로 계측해야 합니다. 텔레메트리 데이터를 CloudWatch로 전달하도록 환경 변수를 구성하고 OpenTelemetry 계측으로 agent를 실행해야 합니다.

프로세스는 다음과 같습니다:

![session_traces](../images/observability_traces.png)

세션 trace가 AgentCore Observability에서 사용 가능해지면 AgentCore Evaluations를 사용하여 agent의 동작을 평가할 수 있습니다.

### Evaluation 레벨
AgentCore Evaluations는 agent 상호작용의 다양한 레벨에서 작동합니다. 세션 정보를 사용하여 전체 대화를 분석할 수 있습니다. trace 정보를 사용하여 대화의 개별 턴에서 사용자 질문에 대한 agent의 응답을 평가할 수도 있습니다. 또는 span 데이터를 사용하여 tool 호출 및 파라미터 선택을 포함한 턴 내부의 정보를 평가할 수 있습니다.

다양한 레벨에 대한 커스텀 메트릭을 생성할 수 있습니다. 내장 메트릭은 다음 범위에서 작동합니다:

![metrics level](../images/metrics_per_level.png)

이 튜토리얼에서는 trace 레벨에서 메트릭을 생성합니다.

### 튜토리얼 결과

이 튜토리얼을 마치면 AgentCore Evaluation의 내장 및 커스텀 메트릭에 대해 배우게 됩니다. 또한 agent의 응답 품질을 측정하는 커스텀 메트릭을 생성하게 됩니다. 


### 사전 요구사항
이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore Starter toolkit

### AgentCore Evaluations 사용하기

Amazon Bedrock AgentCore는 agent와 tool을 개발, 배포 및 모니터링하기 위한 다양한 인터페이스를 지원합니다.

완전한 제어를 위해 [control plane](https://docs.aws.amazon.com/bedrock-agentcore-control/latest/APIReference/Welcome.html) 및 [data plane](https://docs.aws.amazon.com/bedrock-agentcore/latest/APIReference/Welcome.html) API를 사용할 수 있습니다. 이러한 API는 AWS SDK([boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html), [AWS SDK for Java](https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/home.html), [AWS SDK for JavaScript](https://docs.aws.amazon.com/sdk-for-javascript/v3/developer-guide/welcome.html)) 및 AWS Command Line Interface([AWS CLI](https://aws.amazon.com/cli/))와 같은 기타 AWS 개발자 도구를 통해 노출됩니다.

간소화된 접근 방식을 위해 [AgentCore Python SDK](https://github.com/aws/bedrock-agentcore-sdk-python) 및 [AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit)을 사용할 수도 있습니다. AgentCore Python SDK는 agent 개발을 위한 Python 기본 요소를 제공하고 AgentCore Starter Toolkit은 AgentCore 기능을 위한 CLI 도구 및 상위 수준 추상화를 제공합니다.

![agentcore_interfaces](../images/agentcore_interfaces.png)

이 튜토리얼에서는 간소화된 경험을 위해 AgentCore Starter Toolkit을 사용합니다. `03-advanced` 폴더에서 boto3를 직접 사용하는 예제를 찾을 수 있습니다

In [ ]:
# AgentCore Evaluation 클라이언트 생성을 위한 toolkit
from bedrock_agentcore_starter_toolkit import Evaluation
import os
import json
# AWS 세션 및 리전 정보 조회용
from boto3.session import Session

In [ ]:
# 현재 AWS 세션의 리전 정보 조회
boto_session = Session()
region = boto_session.region_name
print(region)

### AgentCore evaluation 클라이언트 초기화하기

이제 evaluation 클라이언트를 초기화하겠습니다. 이 튜토리얼에서는 AgentCore 컴포넌트와의 상호작용을 단순화하여 시작 프로세스를 가속화하는 추상화 SDK인 [AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit)을 사용합니다. 

In [ ]:
# AgentCore Starter Toolkit을 사용한 evaluation 클라이언트 초기화
eval_client = Evaluation(region=region)

### 내장 evaluator 조회하기

이제 사용 가능한 내장 evaluator를 조회하여 어디에 사용할 수 있는지 이해해 보겠습니다. `list_evaluators()` 함수가 이를 도와줄 수 있습니다. 

In [ ]:
# 사용 가능한 내장 evaluator 목록 조회
available_evaluators = eval_client.list_evaluators()
available_evaluators

나중에 온디맨드 및 온라인 평가에서 사용하기 위해 정보를 딕셔너리로 조회할 수도 있습니다:

In [ ]:
# 첫 번째 evaluator의 ID와 설명 출력
print(
    available_evaluators['evaluators'][0]['evaluatorId'], 
    available_evaluators['evaluators'][0]['description']
)

보시다시피 `Builtin.Correctness` 메트릭은 응답 품질을 평가하는 데 도움이 됩니다. 이 메트릭의 세부사항을 이해하기 위해 자세히 살펴보겠습니다. 이를 위해 `get_evaluator` 메서드를 사용할 수 있습니다

In [ ]:
# Builtin.Correctness evaluator의 상세 정보 조회
eval_client.get_evaluator(evaluator_id="Builtin.Correctness")

이 경우 evaluator가 응답을 3개 레벨로 분류하는 것을 볼 수 있습니다: Incorrect, Partially Correct, Correct. 우리의 사용 사례에서는 좀 더 상세하게 5개 레벨 척도를 사용하고 싶습니다. 이를 위해 커스텀 evaluator를 생성할 수 있습니다.

### 커스텀 evaluator 생성하기

이제 5개 레벨 척도를 가질 수 있는 응답 품질을 위한 커스텀 메트릭을 생성해 보겠습니다. 이를 위해 evaluator model을 선택하고, 평가에 대한 지침을 제공하고, 평가 척도를 설정해야 합니다. 이 경우 척도는 Very Good부터 Very Poor까지입니다. JSON 파일에서 평가 구성을 조회해 보겠습니다

In [ ]:
# JSON 파일에서 커스텀 evaluator 설정 로드
with open("metric.json") as f:
    print("Reading custom metric details")
    eval_config = json.load(f)
eval_config

그런 다음 `create_evaluator` 메서드를 사용하여 evaluator를 생성할 수 있습니다. `TRACE` 레벨에서 `response_Quality` evaluator를 생성하겠습니다. 

커스텀 evaluator를 생성할 때 tool call 레벨, trace 레벨 또는 session 레벨에서 적용하도록 선택할 수 있습니다. 

* **tool call**은 agent의 외부 함수, API 또는 기능 호출을 나타내는 span입니다. Tool-call span은 일반적으로 tool 이름, 입력 파라미터, 실행 시간 및 출력과 같은 정보를 캡처합니다. Tool-call 세부사항은 agent가 tool을 올바르고 효율적으로 선택하고 사용했는지 평가하는 데 사용됩니다.

* **trace**는 단일 agent 실행 또는 요청의 완전한 기록입니다. trace는 해당 실행 중에 수행된 개별 작업을 나타내는 하나 이상의 span을 포함합니다. Trace는 agent 결정 및 tool 사용에 대한 엔드투엔드 가시성을 제공합니다.

* **session**은 단일 사용자 또는 워크플로우의 관련 상호작용을 논리적으로 그룹화한 것입니다. session은 하나 이상의 trace를 포함할 수 있습니다. Session은 개별 요청에 집중하는 대신 다단계 상호작용에서 agent 동작을 보고 평가하는 데 도움이 됩니다. 

In [ ]:
# TRACE 레벨에서 작동하는 커스텀 evaluator 생성
custom_evaluator = eval_client.create_evaluator(
    name="response_quality_for_scope",
    level="TRACE",  # trace 레벨: 단일 agent 실행 평가
    description="Response quality evaluator",
    config=eval_config
)

### 다음 튜토리얼을 위한 evaluator 정보 저장하기

이제 다음 튜토리얼에서 사용하기 위해 evaluatorId를 저장하겠습니다. 이를 위해 `evaluator_id` 변수를 저장합니다.

In [ ]:
# 생성된 evaluator의 ID 추출 (다음 튜토리얼에서 사용)
evaluator_id = custom_evaluator['evaluatorId']

In [ ]:
# IPython magic command: 변수를 다음 노트북에서 사용하기 위해 저장
%store evaluator_id

#### 축하합니다

다음 튜토리얼에서 사용할 커스텀 evaluator를 생성했습니다.